### Appendix E: Common Patterns and Idioms

This appendix presents frequently used SystemVerilog patterns and idioms that experienced designers rely on for efficient, readable, and maintainable code.

#### E.1 Clock and Reset Patterns

##### E.1.1 Standard Synchronous Reset
```systemverilog
always_ff @(posedge clk) begin
    if (rst_n == 1'b0) begin
        // Reset all registers to known values
        data_reg <= '0;
        valid_reg <= 1'b0;
        counter <= '0;
    end else begin
        // Normal operation
        if (enable) begin
            data_reg <= data_in;
            valid_reg <= valid_in;
        end
        counter <= counter + 1;
    end
end
```

##### E.1.2 Asynchronous Reset Pattern
```systemverilog
always_ff @(posedge clk or negedge rst_n) begin
    if (rst_n == 1'b0) begin
        data_reg <= '0;
        state <= IDLE;
    end else begin
        data_reg <= next_data;
        state <= next_state;
    end
end
```

##### E.1.3 Clock Domain Crossing (CDC) Pattern
```systemverilog
// Two-flop synchronizer
logic [DATA_WIDTH-1:0] sync_ff1, sync_ff2;

always_ff @(posedge dest_clk or negedge rst_n) begin
    if (!rst_n) begin
        sync_ff1 <= '0;
        sync_ff2 <= '0;
    end else begin
        sync_ff1 <= async_data;
        sync_ff2 <= sync_ff1;
    end
end

assign sync_data = sync_ff2;
```

#### E.2 Finite State Machine Patterns

##### E.2.1 Three-Process FSM Pattern
```systemverilog
typedef enum logic [2:0] {
    IDLE    = 3'b001,
    ACTIVE  = 3'b010,
    DONE    = 3'b100
} state_t;

state_t state, next_state;

// State register
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        state <= IDLE;
    else
        state <= next_state;
end

// Next state logic
always_comb begin
    next_state = state;
    case (state)
        IDLE: if (start) next_state = ACTIVE;
        ACTIVE: if (done_condition) next_state = DONE;
        DONE: if (ack) next_state = IDLE;
        default: next_state = IDLE;
    endcase
end

// Output logic
always_comb begin
    busy = (state != IDLE);
    valid_out = (state == DONE);
    data_out = (state == DONE) ? result_reg : '0;
end
```

##### E.2.2 One-Hot State Machine Pattern
```systemverilog
typedef enum logic [2:0] {
    IDLE_ST   = 3'b001,
    PROC_ST   = 3'b010,
    WAIT_ST   = 3'b100
} onehot_state_t;

onehot_state_t state, next_state;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        state <= IDLE_ST;
    else
        state <= next_state;
end

always_comb begin
    next_state = '0;
    unique case (1'b1)
        state[0]: next_state = start ? PROC_ST : IDLE_ST;
        state[1]: next_state = complete ? WAIT_ST : PROC_ST;
        state[2]: next_state = timeout ? IDLE_ST : WAIT_ST;
        default: next_state = IDLE_ST;
    endcase
end
```

#### E.3 Handshake and Flow Control Patterns

##### E.3.1 Valid-Ready Handshake
```systemverilog
// Producer side
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        valid <= 1'b0;
        data <= '0;
    end else if (!valid || ready) begin
        valid <= new_data_available;
        data <= new_data_available ? next_data : data;
    end
end

// Consumer side
always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        ready <= 1'b1;
    end else begin
        ready <= can_accept_data;
    end
end

// Data transfer occurs when both valid and ready are high
assign transfer = valid && ready;
```

##### E.3.2 Credit-Based Flow Control
```systemverilog
parameter MAX_CREDITS = 8;
logic [$clog2(MAX_CREDITS+1)-1:0] credit_count;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        credit_count <= MAX_CREDITS;
    end else begin
        case ({send_data, receive_credit})
            2'b01: credit_count <= credit_count + 1;
            2'b10: credit_count <= credit_count - 1;
            default: credit_count <= credit_count;
        endcase
    end
end

assign can_send = (credit_count > 0);
```

#### E.4 FIFO and Buffer Patterns

##### E.4.1 Synchronous FIFO Pattern
```systemverilog
parameter DEPTH = 16;
parameter WIDTH = 32;

logic [WIDTH-1:0] memory [DEPTH];
logic [$clog2(DEPTH)-1:0] write_ptr, read_ptr;
logic [$clog2(DEPTH):0] count;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        write_ptr <= '0;
        read_ptr <= '0;
        count <= '0;
    end else begin
        case ({write_en && !full, read_en && !empty})
            2'b01: begin // Read only
                read_ptr <= read_ptr + 1;
                count <= count - 1;
            end
            2'b10: begin // Write only
                memory[write_ptr] <= write_data;
                write_ptr <= write_ptr + 1;
                count <= count + 1;
            end
            2'b11: begin // Read and write
                memory[write_ptr] <= write_data;
                write_ptr <= write_ptr + 1;
                read_ptr <= read_ptr + 1;
            end
        endcase
    end
end

assign read_data = memory[read_ptr];
assign full = (count == DEPTH);
assign empty = (count == 0);
```

##### E.4.2 Gray Code Counter Pattern
```systemverilog
function automatic logic [WIDTH-1:0] bin_to_gray(input logic [WIDTH-1:0] bin);
    return bin ^ (bin >> 1);
endfunction

function automatic logic [WIDTH-1:0] gray_to_bin(input logic [WIDTH-1:0] gray);
    logic [WIDTH-1:0] bin;
    bin[WIDTH-1] = gray[WIDTH-1];
    for (int i = WIDTH-2; i >= 0; i--) begin
        bin[i] = bin[i+1] ^ gray[i];
    end
    return bin;
endfunction

// Gray code counter
logic [WIDTH-1:0] gray_count, next_gray;
logic [WIDTH-1:0] bin_count;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        gray_count <= '0;
    else if (enable)
        gray_count <= next_gray;
end

assign bin_count = gray_to_bin(gray_count);
assign next_gray = bin_to_gray(bin_count + 1);
```

#### E.5 Parameterization Patterns

##### E.5.1 Generic Width Parameterization
```systemverilog
module generic_register #(
    parameter int WIDTH = 32,
    parameter logic [WIDTH-1:0] RESET_VALUE = '0
)(
    input logic clk,
    input logic rst_n,
    input logic enable,
    input logic [WIDTH-1:0] data_in,
    output logic [WIDTH-1:0] data_out
);

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n)
        data_out <= RESET_VALUE;
    else if (enable)
        data_out <= data_in;
end

endmodule
```

##### E.5.2 Interface-Based Parameterization
```systemverilog
interface bus_if #(parameter ADDR_WIDTH = 32, DATA_WIDTH = 32);
    logic [ADDR_WIDTH-1:0] addr;
    logic [DATA_WIDTH-1:0] wdata, rdata;
    logic valid, ready, write_en;
    
    modport master (
        output addr, wdata, valid, write_en,
        input rdata, ready
    );
    
    modport slave (
        input addr, wdata, valid, write_en,
        output rdata, ready
    );
endinterface
```

#### E.6 Error Detection and Handling Patterns

##### E.6.1 Parity Check Pattern
```systemverilog
function automatic logic calc_parity(input logic [WIDTH-1:0] data);
    return ^data; // XOR reduction
endfunction

// Even parity generator
assign parity_bit = calc_parity(data_word);

// Parity checker
logic parity_error;
assign parity_error = calc_parity({data_word, received_parity}) != 1'b0;
```

##### E.6.2 Timeout Pattern
```systemverilog
parameter TIMEOUT_CYCLES = 1000;
logic [$clog2(TIMEOUT_CYCLES+1)-1:0] timeout_counter;
logic timeout_expired;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        timeout_counter <= '0;
        timeout_expired <= 1'b0;
    end else if (start_timer) begin
        timeout_counter <= TIMEOUT_CYCLES;
        timeout_expired <= 1'b0;
    end else if (timeout_counter > 0) begin
        timeout_counter <= timeout_counter - 1;
        timeout_expired <= (timeout_counter == 1);
    end
end
```

#### E.7 Memory Access Patterns

##### E.7.1 Dual-Port RAM Pattern
```systemverilog
module dual_port_ram #(
    parameter DEPTH = 1024,
    parameter WIDTH = 32
)(
    input logic clk,
    
    // Port A
    input logic [$clog2(DEPTH)-1:0] addr_a,
    input logic [WIDTH-1:0] wdata_a,
    output logic [WIDTH-1:0] rdata_a,
    input logic we_a,
    
    // Port B
    input logic [$clog2(DEPTH)-1:0] addr_b,
    input logic [WIDTH-1:0] wdata_b,
    output logic [WIDTH-1:0] rdata_b,
    input logic we_b
);

logic [WIDTH-1:0] memory [DEPTH];

always_ff @(posedge clk) begin
    if (we_a) memory[addr_a] <= wdata_a;
    if (we_b) memory[addr_b] <= wdata_b;
end

assign rdata_a = memory[addr_a];
assign rdata_b = memory[addr_b];

endmodule
```

##### E.7.2 Memory Initialization Pattern
```systemverilog
initial begin
    // Initialize from file
    $readmemh("init_data.hex", memory);
    
    // Or initialize with pattern
    for (int i = 0; i < DEPTH; i++) begin
        memory[i] = i * 2;  // Example pattern
    end
end
```

#### E.8 Assertion Patterns

##### E.8.1 Protocol Checking Patterns
```systemverilog
// Valid-ready protocol check
property valid_ready_protocol;
    @(posedge clk) disable iff (!rst_n)
    valid && !ready |=> valid;
endproperty
assert_valid_stable: assert property(valid_ready_protocol);

// Mutual exclusion check
property mutex_check;
    @(posedge clk) disable iff (!rst_n)
    !(signal_a && signal_b);
endproperty
assert_mutex: assert property(mutex_check);

// FIFO overflow/underflow check
property fifo_no_overflow;
    @(posedge clk) disable iff (!rst_n)
    full |-> !write_en;
endproperty
assert_no_overflow: assert property(fifo_no_overflow);
```

##### E.8.2 Coverage Patterns
```systemverilog
covergroup transaction_cg @(posedge clk);
    cp_cmd: coverpoint command {
        bins read_cmd = {READ};
        bins write_cmd = {WRITE};
        bins idle_cmd = {IDLE};
    }
    
    cp_addr: coverpoint address {
        bins low_addr = {[0:255]};
        bins mid_addr = {[256:511]};
        bins high_addr = {[512:1023]};
    }
    
    cross_cmd_addr: cross cp_cmd, cp_addr;
endgroup

transaction_cg cg_inst = new();
```

#### E.9 Debug and Monitoring Patterns

##### E.9.1 Performance Counter Pattern
```systemverilog
logic [31:0] cycle_counter, transaction_counter;
logic [31:0] utilization_percent;

always_ff @(posedge clk or negedge rst_n) begin
    if (!rst_n) begin
        cycle_counter <= '0;
        transaction_counter <= '0;
    end else begin
        cycle_counter <= cycle_counter + 1;
        if (transaction_active)
            transaction_counter <= transaction_counter + 1;
    end
end

// Calculate utilization (requires division, may need dedicated logic)
assign utilization_percent = (transaction_counter * 100) / cycle_counter;
```

##### E.9.2 Debug Signal Multiplexing
```systemverilog
logic [7:0] debug_select;
logic [31:0] debug_data;

always_comb begin
    case (debug_select)
        8'd0: debug_data = state_register;
        8'd1: debug_data = fifo_count;
        8'd2: debug_data = error_flags;
        8'd3: debug_data = performance_counter;
        default: debug_data = 32'hDEADBEEF;
    endcase
end
```

#### E.10 Best Practices Summary

##### E.10.1 Coding Style Idioms
- Use `always_ff` for sequential logic, `always_comb` for combinational
- Prefer non-blocking assignments (`<=`) in sequential blocks
- Use blocking assignments (`=`) in combinational blocks
- Initialize all registers in reset conditions
- Use meaningful signal names and consistent naming conventions

##### E.10.2 Common Anti-patterns to Avoid
```systemverilog
// DON'T: Mixed blocking and non-blocking in same block
always_ff @(posedge clk) begin
    a <= b;  // Non-blocking
    c = d;   // Blocking - AVOID THIS MIX
end

// DON'T: Unregistered outputs from always_ff
always_ff @(posedge clk) begin
    temp_reg <= input_data;
end
assign output_data = temp_reg + 1;  // Better to register this too

// DON'T: Multiple clocks in sensitivity list
always_ff @(posedge clk1 or posedge clk2) begin  // AVOID
    // Logic here
end
```

##### E.10.3 Synthesis-Friendly Patterns
- Use explicit widths for constants: `4'b0000` instead of `0`
- Avoid latches by ensuring all paths assign to variables
- Use case statements with default clauses
- Prefer `unique case` and `priority case` for synthesis optimization
- Keep critical paths short and well-defined

This appendix serves as a quick reference for commonly used SystemVerilog patterns. These idioms represent battle-tested approaches that promote code reusability, maintainability, and synthesis efficiency.